In [1]:
!pip install --q "snowflake-connector-python[pandas]"
!sudo pip install --q  snowflake-ml-python==1.0.11 -U
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip install --q seaborn
!pip install --q plotly 
!pip install --upgrade --q xgboost==1.7.3
!pip install --upgrade --q numpy==1.24.3
!pip install --upgrade --q pandas==1.5.3
!pip install --upgrade --q anyio==3.5.0
!pip install --upgrade --q packaging==23.1
!pip install --upgrade --q scikit-learn==1.3.0
!pip install --upgrade --q typing-extensions==4.7.1
!pip install --upgrade --q cryptography==39.0.0
!pip install --upgrade --q fsspec==2023.9.2

ERROR: refractio 2.1.5.5 has requirement pandas==2.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: mlflow 2.6.0 has requirement packaging<24, but you'll have packaging 24.0 which is incompatible.
ERROR: mlflow 2.6.0 has requirement pyarrow<13,>=4.0.0, but you'll have pyarrow 16.0.0 which is incompatible.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: snowflake-snowpark-python 1.0.0 has requirement cloudpickle<=2.0.0,>=1.6.0, but you'll have cloudpickle 2.2.1 which is incompatible.
ERROR: snowflake-snowpark-python 1.0.0 has requirement snowflake-connector-python<3.0.0,>=2.7.12, but you'll have snowflake-connector-python 3.10.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement cryptography<39.0.0,>=3.1.0, but you'll have cryptography 42.0.7 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: s

You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: mlflow 2.6.0 has requirement packaging<24, but you'll have packaging 24.0 which is incompatible.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.3.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement itsdangerous==2.0.1, but you'll have itsdangerous 2.1.2 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Jinja2==3.0.3, but you'll have jinja2 3.1.3 wh

ERROR: refractio 2.1.5.5 has requirement pandas==2.0.0, but you'll have pandas 1.5.3 which is incompatible.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have jsonschema 3.2.0 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.3.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement itsdangerous==2.0.1, but you'll have itsdangerous 2.1.2 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Jinja2==3.0.3, but you'll have jinja2 3.1.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement matplotlib==3.6.0; python_version >= "3.8", but you'll have matplotlib 3.7.5 which is incompatible.
ERROR: mosa

ERROR: pyopenssl 24.1.0 has requirement cryptography<43,>=41.0.5, but you'll have cryptography 39.0.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: snowflake-ml-python 1.0.11 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: s3fs 2023.12.2 has requirement fsspec==2023.12.2, but you'll have fsspec 2023.9.2 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install --q statsmodels

ERROR: refractio 2.1.5.5 has requirement pandas==2.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: mlflow 2.6.0 has requirement packaging<24, but you'll have packaging 24.0 which is incompatible.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.11 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have jsonschema 3.2.0 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.3.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement it

In [3]:
import statsmodels

In [4]:
from snowflake.snowpark import Session
# import seaborn as sns
# Pandas Tools
# Data Science Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
import configparser

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

Matplotlib created a temporary cache directory at /tmp/matplotlib-vl667z1e because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [6]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [7]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [8]:
df = session.table("AUTO_INSURANCE_CLAIMS_DATA_PRODUCT")
insurance_claim = df.to_pandas()

In [9]:
insurance_claim.columns = insurance_claim.columns.str.lower()

In [10]:
insurance_claim.columns

Index(['months_as_customer', 'customer_age', 'policy_number',
       'policy_bind_date', 'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital_gains', 'capital_loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day', 'incident_time_of_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount_paid',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported'],
      dtype='object')

In [11]:
insurance_claim.rename(columns = {'total_claim_amount_paid': 'total_claim_amount'}, inplace=True)

In [12]:
insurance_claim=insurance_claim.replace("?",np.NaN)

In [13]:
### changing Incident date into datetime format
insurance_claim['incident_date']=pd.to_datetime(insurance_claim['incident_date'],format='%Y-%m-%d')

### changing auto_year date into datetime format
insurance_claim['auto_year']=pd.to_datetime(insurance_claim['auto_year'],format='%Y')

In [14]:
### Intialize "count" column

insurance_claim['count']=1

In [15]:
#plt.figure(figsize=[5,4])
fig = px.pie(insurance_claim,names='fraud_reported')
fig.write_html("/data/EDA/Fraud_Reported_Pie_Distribution.html")

In [16]:
insurance_claim['fraud_reported'].value_counts()

fraud_reported
No Fraud Reported    199657
Fraud Reported        34608
Name: count, dtype: int64

In [17]:
### Datasplitting into fraud and not fraud category

insurance_claim_f=insurance_claim[insurance_claim['fraud_reported']=='Fraud Reported']
insurance_claim_nf=insurance_claim[insurance_claim['fraud_reported']=='No Fraud Reported']

In [18]:
fig = px.box(insurance_claim, x="policy_state", y="total_claim_amount", color="fraud_reported")
fig.write_html("/data/EDA/Policy_State_vs_FraudClaims.html")

In [19]:
fig=px.scatter(insurance_claim,x='policy_annual_premium',y='total_claim_amount',color='fraud_reported',marginal_x='histogram',marginal_y='histogram',trendline='ols')
fig.write_html("/data/EDA/Policy_Premium_vs_FraudClaims.html")

In [20]:
fig=px.scatter(insurance_claim,x='customer_age',y='total_claim_amount',color='fraud_reported',marginal_x='box',marginal_y='violin',trendline='ols')
fig.write_html("/data/EDA/CustomerAge_vs_FraudClaims.html")

In [21]:
groupdf_is=insurance_claim.groupby(by=['insured_sex','fraud_reported']).count()
groupdf_is.reset_index(inplace=True)
groupdf_is=groupdf_is[['insured_sex','fraud_reported','count']]
groupdf_is.head()

,insured_sex,fraud_reported,count
0,FEMALE,Fraud Reported,17237
1,FEMALE,No Fraud Reported,99573
2,MALE,Fraud Reported,17371
3,MALE,No Fraud Reported,100084


In [23]:
fig=px.bar(groupdf_is,x='insured_sex',y='count',color='fraud_reported',text='count',pattern_shape_sequence=['|','/'],pattern_shape='fraud_reported')
fig.write_html("/data/EDA/InsuredSex_vs_FraudClaims.html")

In [24]:
groupdf_uu=insurance_claim.groupby(by=['umbrella_limit','fraud_reported']).count()
groupdf_uu.reset_index(inplace=True)
groupdf_uu=groupdf_uu[['umbrella_limit','fraud_reported','count']]
groupdf_uu.head(3)

,umbrella_limit,fraud_reported,count
0,0,Fraud Reported,4084
1,0,No Fraud Reported,27015
2,2000003,No Fraud Reported,4


In [25]:
fig=px.bar(groupdf_uu,x='umbrella_limit',y='count',color='fraud_reported',text='count')
fig.write_html("/data/EDA/UmbrellaLimit_vs_FraudClaims.html")

In [26]:
groupdf_el=insurance_claim.groupby(by=['insured_education_level','fraud_reported']).count()
groupdf_el.reset_index(inplace=True)
groupdf_el=groupdf_el[['insured_education_level','fraud_reported','count']]
groupdf_el.head()

,insured_education_level,fraud_reported,count
0,Associate,Fraud Reported,4993
1,Associate,No Fraud Reported,29157
2,College,Fraud Reported,4328
3,College,No Fraud Reported,23975
4,High School,Fraud Reported,5100


In [27]:
fig=px.bar(groupdf_el,x='insured_education_level',y='count',color='fraud_reported',text='count',title="Insurance Claim for different education Level")
fig.write_html("/data/EDA/InsuredEducationLevel_vs_FraudClaims.html")

In [28]:
groupdf_ic=insurance_claim.groupby(by=['insured_occupation','fraud_reported']).count()
groupdf_ic.reset_index(inplace=True)
groupdf_ic=groupdf_ic[['insured_occupation','fraud_reported','count']]
groupdf_ic.head()

,insured_occupation,fraud_reported,count
0,adm-clerical,Fraud Reported,2330
1,adm-clerical,No Fraud Reported,12980
2,armed-forces,Fraud Reported,2395
3,armed-forces,No Fraud Reported,13700
4,craft-repair,Fraud Reported,2106


In [29]:
fig=px.bar(groupdf_ic,x='insured_occupation',y='count',color='fraud_reported',text='count',title='Number of Claims diffrent Occupation')
fig.write_html("/data/EDA/InsuredOccupation_vs_FraudClaims.html")

In [30]:
groupdf_h=insurance_claim.groupby(by=['insured_hobbies','fraud_reported']).count()
groupdf_h.reset_index(inplace=True)
groupdf_h=groupdf_h[['insured_hobbies','fraud_reported','count']]
groupdf_h.head()

,insured_hobbies,fraud_reported,count
0,base-jumping,Fraud Reported,1663
1,base-jumping,No Fraud Reported,9731
2,basketball,Fraud Reported,1756
3,basketball,No Fraud Reported,10207
4,board-games,Fraud Reported,1765


In [31]:
fig=px.bar(groupdf_h,x='insured_hobbies',y='count',color='fraud_reported',text='count',title="Hobby Wise claim")
fig.write_html("/data/EDA/InsuredHobbies_vs_FraudClaims.html")

In [32]:
groupdf_rl=insurance_claim.groupby(by=['insured_relationship','fraud_reported']).count()
groupdf_rl.reset_index(inplace=True)
groupdf_rl=groupdf_rl[['insured_relationship','fraud_reported','count']]
groupdf_rl.head()

,insured_relationship,fraud_reported,count
0,husband,Fraud Reported,5669
1,husband,No Fraud Reported,32950
2,not-in-family,Fraud Reported,5653
3,not-in-family,No Fraud Reported,33389
4,other-relative,Fraud Reported,5837


In [33]:
fig=px.bar(groupdf_rl,x='insured_relationship',y='count',color='fraud_reported',text='count')
fig.write_html("/data/EDA/InsuredRelationship_vs_FraudClaims.html")

In [34]:
groupdf_it=insurance_claim.groupby(by=['incident_type','fraud_reported']).count()
groupdf_it.reset_index(inplace=True)
groupdf_it=groupdf_it[['incident_type','fraud_reported','count']]
groupdf_it.head()

,incident_type,fraud_reported,count
0,Multi-vehicle Collision,Fraud Reported,11496
1,Multi-vehicle Collision,No Fraud Reported,62829
2,Parked Car,Fraud Reported,2201
3,Parked Car,No Fraud Reported,19053
4,Single Vehicle Collision,Fraud Reported,20747


In [35]:
fig=px.bar(groupdf_it,x='incident_type',y='count',color='fraud_reported',text='count',title="Number of claims Vs. Incident type")
fig.write_html("/data/EDA/IncidentType_vs_FraudClaims.html")

In [36]:
groupdf_ct=insurance_claim.groupby(by=['collision_type','fraud_reported']).count()
groupdf_ct.reset_index(inplace=True)
groupdf_ct=groupdf_ct[['collision_type','fraud_reported','count']]
groupdf_ct.head()

,collision_type,fraud_reported,count
0,Details not Available,Fraud Reported,4699
1,Details not Available,No Fraud Reported,29056
2,Front Collision,Fraud Reported,8573
3,Front Collision,No Fraud Reported,48745
4,Rear Collision,Fraud Reported,12736


In [37]:
fig=px.bar(groupdf_ct,x='collision_type',y='count',color='fraud_reported',text='count',title="Nos. of Claim for different type of collision")
fig.write_html("/data/EDA/CollisionType_vs_FraudClaims.html")

In [38]:
groupdf_ins=insurance_claim.groupby(by=['incident_severity','fraud_reported']).count()
groupdf_ins.reset_index(inplace=True)
groupdf_ins=groupdf_ins[['incident_severity','fraud_reported','count']]
groupdf_ins.head()

,incident_severity,fraud_reported,count
0,Major Damage,Fraud Reported,18692
1,Major Damage,No Fraud Reported,113032
2,Minor Damage,Fraud Reported,9616
3,Minor Damage,No Fraud Reported,54832
4,Total Loss,Fraud Reported,6300


In [39]:
fig=px.bar(groupdf_ins,x='incident_severity',y='count',color='fraud_reported',text='count',title="Number of claim Vs. Incident severity")
fig.write_html("/data/EDA/IncidentSeverity_vs_FraudClaims.html")

In [40]:
groupdf_ac=insurance_claim.groupby(by=['authorities_contacted','fraud_reported']).count()
groupdf_ac.reset_index(inplace=True)
groupdf_ac=groupdf_ac[['authorities_contacted','fraud_reported','count']]
groupdf_ac.head()

,authorities_contacted,fraud_reported,count
0,Ambulance,Fraud Reported,6849
1,Ambulance,No Fraud Reported,39504
2,Fire,Fraud Reported,6987
3,Fire,No Fraud Reported,39702
4,None,Fraud Reported,7057


In [41]:
fig=px.bar(groupdf_ac,x='authorities_contacted',y='count',color='fraud_reported',text='count')
fig.write_html("/data/EDA/AuthoritiesContacted_vs_FraudClaims.html")

In [42]:
groupdf_st=insurance_claim.groupby(by=['incident_state','fraud_reported']).count()
groupdf_st.reset_index(inplace=True)
groupdf_st=groupdf_st[['incident_state','fraud_reported','count']]
groupdf_st.head()

,incident_state,fraud_reported,count
0,CT,Fraud Reported,34608
1,CT,No Fraud Reported,199657


In [43]:
groupdf_atmm=insurance_claim.groupby(by=['auto_make','auto_model','fraud_reported']).count()
groupdf_atmm.reset_index(inplace=True)
groupdf_atmm=groupdf_atmm[['auto_make','auto_model','fraud_reported','count']]
groupdf_atmm['fraud_reported']=groupdf_atmm['fraud_reported'].apply(lambda x: 'Fraud Reported :YES' if x == 'Y' else 'Fraud Reported :NO')
groupdf_atmm.head()

,auto_make,auto_model,fraud_reported,count
0,Accura,MDX,Fraud Reported :NO,994
1,Accura,MDX,Fraud Reported :NO,5489
2,Accura,RSX,Fraud Reported :NO,552
3,Accura,RSX,Fraud Reported :NO,3509
4,Accura,TL,Fraud Reported :NO,685


In [ ]:
fig=px.bar(groupdf_atmm,x='auto_make',y='count',color='auto_model',text='count',hover_name='fraud_reported',title="Claims by Auto make amd Auto model")
fig.show()